<a href="https://colab.research.google.com/github/Marie1009/PIR/blob/master/google_collaboratory/HCDAtoWinepi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Algorithmes principaux : groupPaires, biggerGroup, winepi**


------------------------------------------------------------------------
### **Algorithme groupPaires**

**Entrée :**

  Séquence des paires fréquentes trouvées par HCDA
      
      format: 
      [[t1, t2], ['A', 'B'], fAB, LtAB]

     t1,t2 intervalle d'apparition de l'épisode A par rapport à l'épisode B
     fAB fréquence d'apparition de la paire AB 
     LtAB : liste des timestamps de A et B 
        par exemple : 
        LtAB=[[TA1, TB1],[TA2,TB2],...]
        où TA1 est un timestamp de A dans la séquence

**But :**
Regrouper les paires d'épisodes (chronique de taille 2) pour générer des chroniques de taille 3.
Ces chroniques seront exploitées par l'algorithme biggerGroup pour former des chroniques de taille 4 et plus.

**Sortie**:

    format:
    [['A','B','C'],f,[[t1, t2], ('A', 'B')],[[t1, t2], ('B', 'C')],[[t1, t2], ('A', 'C')] ]
    premier élément : chronique de taille 3 trouvée
    deuxième élément : fréquence d'apparition de cette chronique de taille 3
    élément 3,4 et 5 : chroniques de taille 2 qui appartiennent à la chronique de taille 3 avec leur intervalle d'apparition

In [0]:
def findPaire(element1,element2,SequenceE,j) :
  """Trouve les paires d'éléments composées qui de [element1,element2]"""
  result=[]
  if (element1>element2) :
    aux=element1                            #Tri element1 et element2 par ordre alphabétique
    element1=element2
    element2=aux
  for i in range(j+1,len(SequenceE)) :      
    Paire=SequenceE[i]
    if ((Paire[1][0]==element1) and (Paire[1][1]==element2)) :
      result.append(Paire)
  return result

In [0]:
def indexOf(Paire,element) :
  """Renvoie l'index de element dans la liste Paire"""
  index = -1
  if(Paire[1][0]==element):
    index = 0
  elif(Paire[1][1]==element):
    index = 1
  else :
    index = 2
  return(index)


In [0]:
def elementCom(Paire1,Paire2) :
  """renvoie elementCommun entre Paire1 et Paire2 , elementPaire1 qui est l'autre element de Paire1, elementPaire2 qui est l'autre element de Paire2
  Si aucun element en commun renvoyer ' ' pour elementCommun, si elem_com = '_' , le reste de la sortie n'a pas de sens"""
  elem_com=" "
  elementPaire1=" "
  elementPaire2=" "
  p1 = Paire1[1]
  p2 = Paire2[1]
  if p1 != p2 :
      for i in range (2):
          for j in range (2):
              if p1[i] == p2[j]:
                  elem_com = p1[i]      
                  elementPaire1 = p1[1-i]    
                  elementPaire2 = p2[1-j]     
  return elem_com, elementPaire1, elementPaire2

In [0]:
def groupPaires(SequenceE,freqMin):
  """Regrouper les paires d'épisodes (chronique de taille 2) pour générer des chroniques de taille 3. Ces chroniques seront exploitées par l'algorithme biggerGroup pour former des chroniques de taille 4 et plus."""
  result=[]
  for i in range(len(SequenceE)) :
    Paire1=SequenceE[i]
    #in sequenceE\Paire1
    for j in range(i+1,len(SequenceE)) :
      Paire2=SequenceE[j]
      elementCommun, elementPaire1, elementPaire2=elementCom(Paire1,Paire2)
      if(elementCommun[0]!= ' '):
        #on cherche la paire pour completer le triplet en partant de j pour ne pas compter des triplets plusieurs fois
        ListPaire3=findPaire(elementPaire1,elementPaire2,SequenceE,j)
        #pour chaque paire correspondante, on calcul la frequence du triplet
        for Paire3 in ListPaire3: 
          NBmatch=0
          for paire1Occ in Paire1[3] :
            for paire2Occ in Paire2[3] :
              #on regarde les occurences des evenements de chaque paire pour voir si c'est les memes 
              #si [a,b] et [b,c], on regarde si le b en question est le meme (meme index dans la sequence)
              if (paire1Occ[indexOf(Paire1,elementCommun)]==paire2Occ[indexOf(Paire2,elementCommun)]) :
                for paire3Occ in Paire3[3] :
                  if (paire1Occ[indexOf(Paire1,elementPaire1)]==paire3Occ[indexOf(Paire3,elementPaire1)] and paire2Occ[indexOf(Paire2,elementPaire2)]==paire3Occ[indexOf(Paire3,elementPaire2)]) :
                    NBmatch+=1
          #enregistrer le triplet si il est assez frequent
          if (NBmatch>=freqMin) :
            result.append([[elementPaire1,elementPaire2,elementCommun],NBmatch,Paire1,Paire2,Paire3])
  return result
                    
            


### **Algorithme biggerGroup**

**Entrée :**

  Séquence des chroniques de taille 3 issues de groupPaires
      
    format:
    [['A','B','C'],f,[[t1, t2], ('A', 'B')],[[t1, t2], ('B', 'C')],[[t1, t2], ('A', 'C')] ]
    premier élément : chronique de taille 3 trouvée
    deuxième élément : fréquence d'apparition de cette chronique de taille 3
    élément 3,4 et 5 : chroniques de taille 2 qui appartiennent à la chronique de taille 3 avec leur intervalle d'apparition

**But :**
Regrouper les chroniques de taille 3 pour générer des chroniques de taille 4 et plus. Toutes les chroniques (de taille 2 à +INF) seront exploitées par l'algorithme Winepi.

**Sortie :**

Séquence de chroniques de taille 4 et plus.

    format:
    [['A', 'C', 'B', 'D'], f, 
      [[t1,t2], ['A', 'B'], fAB, LtAB], 
      [[t1,t2], ['B', 'C'], fBC, LtBC], 
      [[t1,t2], ['A', 'C'], fAC, LtAC], 
      [[t1,t2], ['A', 'D'], fAD, LtAD], 
      [[t1,t2], ['B', 'D'], fBD, LtBD], 
      [[t1,t2], ['C', 'D'], fCD, LtCD]]

    premier élément : liste des événements de la chronique 
    deuxième élement : fréquence d'apparition de cette chronique
    élément 3,4, ... : Paires faisant partie de cette chronique 
 

    

In [0]:
def get_occurence_chronicle(chronicle,element):
  """return the list of timestamps of element in chronicle"""
  i=2
  Found=0
  #find the first sub-chronicle in chronicle having element 
  while(i<len(chronicle) and Found!=1):
      if(element in chronicle[i][1]) :
        occElementChronicle=[]
        #index element in the paire : 0 or 1
        indexElement=indexOf(chronicle[i],element)
        #create the list of timestamps
        for j in range(len(chronicle[i][3])) :
            occElementChronicle.append(chronicle[i][3][j][indexElement])                 
        Found=1
      else:
        i+=1
  return occElementChronicle

def get_occurence_paire(paire,element):
  """return the list of timestamps of element in paire"""
  occElementPaire=[]
  #index element in the paire : 0 or 1
  index=indexOf(paire,element)
  #paire[3] : interval occurences of the paire
  for j in range(len(paire[3])) :
      occElementPaire.append(paire[3][j][index])                 
  return occElementPaire

def count_similar_occurence(listOccurence1, listOccurence2):
  """return the number of similar occurence between listOccurence1 and listOccurence2 for one event"""
  nbMatch=0
  for timestamp in listOccurence1:
    if(timestamp in listOccurence2):
      nbMatch+=1
  return nbMatch

def match_occurences(chronicle1,chronicle2,paire,count) :
  """return the number of match occurences for each elements of the list of events count between chronicle1, chronicle2 and paire"""
  listOccurencesMatch=[]
  for element in count:
    #chronicle[0] : events in the chronicle
    if element in chronicle1[0] :
      occElementChronicle1=get_occurence_chronicle(chronicle1,element)
      numChronicle=1
    if element in chronicle2[0] :
      occElementChronicle2=get_occurence_chronicle(chronicle2,element)
      numChronicle=2
    #if element is only in one of the two chronicles
    if count[element]<2 : 
      occElementPaire=get_occurence_paire(paire,element)
      if(numChronicle==1) :
        nbMatch=count_similar_occurence(occElementChronicle1,occElementPaire)
      if(numChronicle==2) :
        nbMatch=count_similar_occurence(occElementChronicle2,occElementPaire)
    #if element is in the two chronicles
    if count[element]==2 : 
      nbMatch=count_similar_occurence(occElementChronicle1,occElementChronicle2)
    listOccurencesMatch.append([element,nbMatch])
  return listOccurencesMatch


In [0]:
def build_group(similarities,group1,group2,Paire,NBmatch,count):
  """retourne le resultat sous le format de sortie de biggerGroup"""
  groupBuilt=[]
  event=[]
  #on ajoute la liste des evenements
  for x in count :
    event.append(x)
  groupBuilt.append(event)
  groupBuilt.append(NBmatch)
  #on ajoute toutes les paires de la premiere chronique
  for p in group1[2:] :
    groupBuilt.append(p)
  #on enleve les paires communes aux deux de la deuxieme chronique 
  for p in similarities:
    group2.pop(group2.index(p))
  #on ajoute les restantes : celles uniques a la deuxieme chronique
  for p in group2[2:] :
    groupBuilt.append(p)
  #on ajoute la paire qui les relie
  groupBuilt.append(Paire)
  return groupBuilt

In [0]:
def biggerGroup(SequencePaires,SequenceGroups,freqmin):
  count={}
  result=[]
  for i in range(len(SequenceGroups)):
    group1=SequenceGroups[i]
    for j in range(i+1,len(SequenceGroups)):
      group2=SequenceGroups[j]
      #check si les deux chroniques regardées ont toutes leurs paires en commun sauf deux
      similarities=[]
      for p1 in range(2,len(group1)):
        for p2 in range(2,len(group2)):
          if (group1[p1][0]==group2[p2][0] and group1[p1][1]==group2[p2][1]) :
            similarities.append(group2[p2])
      if (len(similarities)==len(group1)-4) :
        #fais la liste des evenements des deux chroniques et indique si ils aparraissent dans une seule (count=1 -> aloneElem) ou les deux (count=2)
        count.clear()
        for a in group1[0]:
          count[a]=1
        for a in group2[0]:
          if a in count:
            count[a]=2
          else :
            count[a]=1
        if len(count)==len(group1)+1-2 :
          aloneElem=[]
          for x, y in count.items() :
            if y==1 :
              aloneElem.append(x)
          #on cherche toutes les paires qui permettent de relier les deux éléments solitaires
          possiblePaire=findPaire(aloneElem[0],aloneElem[1],SequencePaires,0)
          #on regarde les occurences des evenements de chaque paire pour voir si c'est les memes 
          #ex : si [a,b] et [b,c], on regarde si le b en question est le meme (meme index dans la sequence)
          for Paire in possiblePaire:
            NBmatch=match_occurences(group1,group2,Paire,count)
            totalMatch=0
            matching=1
            for match in NBmatch :
              if match[1]<freqmin:
                matching=0
              totalMatch+=match[1]
            #on ajoute la nouvelle chronique composée des deux chroniques et de la paire si elle satisfait la frequence minimale
            if matching==1:
              totalMatch=totalMatch/len(count)
              new_group=build_group(similarities,group1,group2,Paire,totalMatch,count)
              result.append(new_group)
  return result


### **Algorithme Winepi**


---

**Entrée :**

# Séquence de chroniques de taille 2 à +INF et non des triplets ! 
# A modifier !!



**Entrée :**
    

**But :**


**Sortie :**


 

    

In [0]:
def winepi2(sequence,paires,conf_min) :
  """Création des règles de l'algorithme Winepi à partir du seuil minimal de confdence conf_min, rules type event->paire"""
  freq={}
  for e in sequence:
    if e[0] in freq :
      freq[e[0]]+=1
    else :
      freq[e[0]]=1
  rules=[]
  for p in paires:
    for e in p[1]:
      conf=p[2]/freq[e]
      if conf>= conf_min :
        rules.append([e,p,conf])
  return rules

In [0]:
def winepi(chronique, conf_min) :
  """Création des règles de l'algorithme Winepi à partir du seuil minimal de confdence conf_min, rules type paire->chronique"""
  rules=[]
  for trip in chronique:
    for Paire in trip[2:] :
      conf=trip[1]/Paire[2]
      if conf>= conf_min :
        rules.append([Paire,trip,conf])
  return rules

In [0]:
def get_rules(Paire, rules):
  found_rules=[]
  for r in rules:
    if Paire==r[0] :
      found_rules.append(r) 
  return found_rules

In [0]:
def get_info_paire (paire) :
  """Renvoie les 2 éléments d'une paire"""
  return(paire[0]+paire[1])

def print_event_rules (rules):
  for rule in rules:
    print(rule[0],end=" ")
    print(" -> ",end=" ")
    print(rule[1][0],end=" ")
    print(rule[1][1],end=" ")
    print(" with conf ",end=" ")
    print(rule[2],end =" ")
    if len(rule)>3 :
      print(" success rate : ",rule[3][0],"/",rule[3][1],end =" ")
    print(" ")
  return None

def print_big_rules (rules) :
  """Affichage propre des règles précedemment crées avec winepi"""
  for rule in rules:
    print(rule[0][0],end=" ")
    print(rule[0][1],end=" ")
    print(" -> ",end=" ")
    print(rule[1][0],end=" ")
    print(" with conf ",end=" ")
    print(rule[2],end =" ")
    if len(rule)>3 :
      print(" success rate : ",rule[3][0],"/",rule[3][1],end =" ")
    print(" ")
  return None

In [0]:
def associate_with_paire(event1,event2,ecart,listPaires) :
  """find the frequent paires (0 or many) known associated with the paire [event1,event2], ecart """
  #ordonne par ordre alphabetique
  if event1 > event2 :
    aux=event1
    event1=event2
    event2=aux
    ecart=-ecart
  matching_paires=[]
  for p in listPaires :
    if p[1]==(event1,event2) and p[0][0]<=ecart<=p[0][1] :
      matching_paires.append(p)
  #if matching_paires != []:
  #  print("searching paire ",event1," ",event2," ecart ",ecart)
  #  print(matching_paires)
  return matching_paires

In [0]:
def predict_event2(rule,event):
  """predict event coming for rule of type event->paire"""
  if event[0]==rule[1][1][0]:
    min_t=event[1]+rule[1][0][0]
    max_t=event[1]+rule[1][0][1]
    next_event=rule[1][1][1]
  else:
    min_t=event[1]-rule[1][0][1]
    max_t=event[1]-rule[1][0][0] 
    next_event=rule[1][1][0] 
  return [next_event,[min_t,max_t],rule[2],rule]

In [0]:
def predict_event(rule, event1, event2):
  """predict event coming for rule of type paire->triplet"""
  max_t=float('inf')
  min_t=float('-inf')
  if event1[0] > event2[0] :
    aux=event1
    event1=event2
    event2=aux
  for p in rule[1][2:] :
    if p[1]!=(event1[0],event2[0]) :
      if p[1][0]==event1[0] :
        min_t = max(min_t,event1[1]+p[0][0])
        max_t = min(max_t,event1[1]+p[0][1])
      if p[1][1]==event1[0] :
        min_t = max(min_t,event1[1]-p[0][1])
        max_t = min(max_t,event1[1]-p[0][0])
      if p[1][0]==event2[0] :
        min_t = max(min_t,event2[1]+p[0][0])
        max_t = min(max_t,event2[1]+p[0][1])
      if p[1][1]==event2[0] :
        min_t = max(min_t,event2[1]-p[0][1])
        max_t = min(max_t,event2[1]-p[0][0])
  for e in rule[1][0]:
    if e!=event1[0] and e!=event2[0] :
      next_event=e
  return [next_event,[min_t,max_t],rule[2],rule]

In [0]:
def verif_prediction(sequence, event) :
  """check if the prediction is accurate"""
  found=False
  for e in sequence:
    if event[1][1]<e[1]:
      return found
    if e[0]==event[0]:
      if event[1][0]<=e[1]<=event[1][1]:
        found=True
        return found
  return found

In [0]:
def accuracy(prediction,sequence,rules) :
  for r in rules:
    r.append([0,0])
  for e in prediction :
    found=verif_prediction(sequence,e)
    i=rules.index(e[3])
    rules[i][3][1]+=1
    if found :
      rules[i][3][0]+=1
  return rules

In [0]:
def prediction2(sequence, listPaires, rules) :
  """predict and compute accuracy of rules of type event->paire"""
  prediction=[]
  for e in sequence:
    for r in rules :
      if r[0]==e[0]:
        next_event=predict_event2(r,e)
        prediction.append(next_event)
  return prediction

In [0]:
def prediction(sequence, listPaires, rules,ecart) :
  """predict and compute accuracy of rules of type paire->triplet"""
  prediction=[]
  previous_events=[]
  for e in sequence:
    for j in previous_events:
      if e[1]-j[1] <= ecart :
        possible_paires=associate_with_paire(j[0],e[0],e[1]-j[1],listPaires)
        #if possible_paires!= [] :
        #  print("searching paire ",j[0]," index ",sequence.index(j)," ",e[0], " index ",sequence.index(e)," ecart ",e[1]-j[1])
        #  print(possible_paires)
        for p in possible_paires:
          r=get_rules(p,rules)
          #predict event -> return next_event, [t1,t2], conf (l'event next_event arrivera entre t1 et t2, avec une conf )
          for rule in r:
            next_event=predict_event(rule,e,j)
            prediction.append(next_event)
      else :
        previous_events.remove(j)
    previous_events.append(e)
  return prediction

In [0]:
def analyse_result(rules,tolerance):
  num_rules_ok=0
  num_rules_ko=0
  num_rules_better=0
  avg_gap=0 #conf - tested_accuracy
  for r in rules :
    if r[3][1] > 0 :
      if (r[2]-tolerance) <= (r[3][0]/r[3][1]) <=(r[2]+tolerance) :
        num_rules_ok +=1
      elif (r[2]+tolerance) <= (r[3][0]/r[3][1]):
        num_rules_better +=1
      else :
        num_rules_ko +=1
      avg_gap+= abs(r[2]-(r[3][0]/r[3][1]))
  avg_gap=avg_gap/(num_rules_ok+num_rules_ko+num_rules_better)
  return num_rules_ok, num_rules_ko, num_rules_better, avg_gap

Affichage des résultats


---



In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.chdir("/content/gdrive/My Drive/PIR/Reconnaissance et Apprentissage de Chroniques")
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
'Anciens fichiers .ipynb'		  main_final.py
 ccdc_final.py				  main_upgrade.py
 CCDC.ipynb				  __pycache__
 ccdc.py				 'suivi travail.gdoc'
 CCDC_sliding.ipynb			  test_dataset_HCDA.ipynb
 ccdc_upgrade.py			  test_main.ipynb
 datasetP				  test_main_marie.ipynb
 Fichiers				  test_main_upgrade.ipynb
 final_main_hcda.ipynb			 'test_sans_py -- V4_HCDA.ipynb'
 HCDA.ipynb				  Untitled0.ipynb
 hcda.py				  v4_hcda_final.py
 HCDA_tester.ipynb			  V4_HCDA.ipynb


In [0]:
#séquence de test
#Paires=[[[2,3], ['A', 'B'], 3, [[1, 2],[5,6],[10,11]]],[[1,2], ['B', 'C'], 3, [[2, 3],[6,7],[11,12]]],[[3,5], ['A', 'C'], 3, [[1, 3],[5,7],[10,12]]],[[4,6], ['A', 'D'], 3, [[1, 4],[5,14],[10,16]]],[[2,3], ['B', 'D'], 3, [[2, 4],[6,14],[11,16]]],[[1,1], ['C', 'D'], 3, [[3, 4],[7,14],[12,16]]],[[1,2], ['E', 'F'], 1, [[4, 13],[14,15],[16,17]]]]
#print("--------Test sequence: --------------")
#print(Paires)
#dataTest=[["A",2],["B",4],["C",5],["D",6],["D",7],["A",8],["B",11],["C",13],["D",14],["B",15],["E",15],["A",16],["C",18],["D",17],["B",19],["F",22]]
#print("--------Test data: --------------")
#print(dataTest)

#séquence dataset salade à partir de HCDA
#import hcda_return
#import ccdc
#limite_ecarts = 1000000000
#sequences=hcda_return.get_sequences("/content/gdrive/My Drive/PIR/Reconnaissance et Apprentissage de Chroniques/datasetP")
#for s in sequences:
# print(sequences.index(s),s)

#sequenceHCDA=hcda_return.main(limite_ecarts,lambda x: ccdc.CCDC(x,limite_ecarts),"/content/gdrive/My Drive/PIR/Reconnaissance et Apprentissage de Chroniques/datasetP")
#print("--------Sequence : --------------")
#print(sequences[40])
#print("--------HCDA : --------------")
#print(sequenceHCDA[3])



#import ccdc_upgrade as ccdc
#import hcda_tester as hcda
import ccdc_final as ccdc
import v4_hcda_final as hcda

import time

# Debut du decompte du temps
start_time = time.time()

# recuperation des datasets
all_data = hcda.get_sequences("/content/gdrive/My Drive/PIR/Reconnaissance et Apprentissage de Chroniques/datasetP")
print('get_sequences')
# modification des timestamps sans concatenation des sequences (division par 1000 de toutes les valeurs)
data_modif = hcda.modif_timestamps(all_data,0.001)
print('modif_timestamps')

# choix du nombre de sequences a analyser
nb_salades = 40
data_selected = data_modif[:nb_salades]
#concatenation et decalage des timestamps
transformed_data= hcda.decal_concat(data_selected)
print('decal_concat')
#for d in transformed_data:
#    print(d)
nb_test=14
data_for_test = data_modif[nb_salades:nb_salades+nb_test]
concat_data_for_test = hcda.decal_concat(data_for_test)

limite_ecarts = hcda.exploration(data_selected)
limite_freq = nb_salades #nb_salades #au minimum on veut que les chroniques soient presentes 1 fois par salade... mais pas verifiable

gaps_dict = {}
for E in data_selected:
    ccdc.get_gaps(E, limite_ecarts, gaps_dict)
base = ccdc.generate_trees(gaps_dict)
#ccdc.print_tree(base)

#base = ccdc.CCDC(transformed_data,limite_ecarts)
#ccdc.print_tree(base)

#exit()
results = hcda.main_concat_v2(limite_ecarts,limite_freq,base,transformed_data)
#results = v4_hcda.main_concat(limite_ecarts,limite_freq,lambda x: ccdc.CCDC(x,limite_ecarts),transformed_data)

#avec indice
#results = v4_hcda.main_concat(limite_ecarts,limite_freq,lambda x: ccdc.CCDC(x,limite_ecarts),transformed_data[:v4_hcda.indice(nb_salades,all_data)])

# Affichage du temps d execution
print("Temps d execution : %s secondes ---" % (time.time() - start_time))

def affichage(sequenceRules,sequenceData,Paires,ecart,tolerance) :
  """sequenceRules is used to create rules, sequencePaires too, sequenceData is used to test the rules"""
  print("-------------sequenceData---------------")
  print(sequenceData)
  #groupPaires
  groupPairesResult=groupPaires(Paires,10)
  #print("--------Group Paires: --------------")
  #print(groupPairesResult)
  
  #biggerGroup
  #biggerGroupResult=biggerGroup(sequence,groupPairesResult,2)
  #print("--------Bigger group : --------------")
  #print(biggerGroupResult)

  #winepi
  winepiResultGroupPaire=winepi(groupPairesResult,0.6)
  #print("--------Winepi group paire : --------------")
  #print_big_rules(winepiResultGroupPaire)

  #winepiResult=winepi(biggerGroupResult,0.8)
  #print("--------Winepi bigger Group : --------------")
  #print_rules(winepiResult)
  
  
  print("\n")
  print("-------event->paire----------")
  winepiResult=winepi2(sequenceRules,Paires,0.6)
  predict_e=prediction2(sequenceData,Paires,winepiResult)
  #for p in predict_e :
  #  print(p)
  #print("\n")
  rules=accuracy(predict_e,sequenceData,winepiResult)
  n_ok,n_ko,n_bet,avg_gap =analyse_result(rules,tolerance)
  #print_event_rules(rules)
  print("rules ok : ",n_ok," rules under : ",n_ko," rules over : ",n_bet," average gap : ",avg_gap)
  
  
  
  #print("--------Winepi group paire : --------------")
  #print_rules(winepiResult)

  #winepiResult=winepi(biggerGroupResult,0.8)
  print("--------Winepi bigger Group : --------------")
  #print_rules(winepiResult)
  print("\n")
  print("--------Prédiction  : paire-> triplet --------------")
  predict_p=prediction(sequenceData,Paires,winepiResultGroupPaire,ecart)
  #for p in predict_p :
  #  print(p)
  #print("\n")
  rules=accuracy(predict_p,sequenceData,winepiResultGroupPaire)
  #print_big_rules(rules)
  print("\n")
  n_ok,n_ko,n_bet,avg_gap =analyse_result(rules,tolerance)
  print("rules ok : ",n_ok," rules under : ",n_ko," rules over : ",n_bet," average gap : ",avg_gap)
  print("\n\n")

  return None

#affichage(dataTest,dataTest,sequenceData,5)
#affichage(sequences[9],sequences[9],sequenceHCDA[1],9000000)
#affichage(sequences[40],sequences[40],sequenceHCDA[3],90000000)

affichage(transformed_data,concat_data_for_test,results,limite_ecarts,0.02)
#affichage(transformed_data,data_modif[nb_salades+2],results,limite_ecarts,0.05)
#affichage(transformed_data,data_modif[nb_salades+3],results,limite_ecarts,0.05)
#affichage(transformed_data,data_modif[nb_salades+4],results,limite_ecarts,0.05)
#affichage(transformed_data,data_modif[nb_salades+5],results,limite_ecarts,0.05)

get_sequences
modif_timestamps
timestamp le plus grand : 1643364
ecart min entre timestamps : 166
ecart max entre timestamps : 81211
1643364
decal_concat
timestamp le plus grand : 579811
ecart min entre timestamps : 200
ecart max entre timestamps : 76477
579811
timestamp le plus grand : 1643364
ecart min entre timestamps : 166
ecart max entre timestamps : 81211
time constraint    events                                                                frequence  occurences
-----------------  ------------------------------------------------------------------  -----------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------